In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import seaborn as sn
import os
from tqdm.notebook import tqdm
from urllib.error import HTTPError

plt.style.use("ggplot")

In [2]:
# Read tidied regine data
reg_year = 2022
data_fold = f"/home/jovyan/shared/teotil3/core_data_june_{reg_year}"
teo_gpkg = os.path.join(data_fold, "tidied", "teotil3_data.gpkg")
reg_gdf = gpd.read_file(
    teo_gpkg, layer=f"regine_{reg_year}", driver="GPKG"
)

# Get just relevant columns
cols = [
    "regine",
    "vassom",
    "geometry",
]
reg_gdf = reg_gdf[cols]
reg_gdf.head()

,regine,vassom,geometry
0,001.10,001,"MULTIPOLYGON (((297006.830 6543966.950, 297169..."
1,001.1A1,001,"MULTIPOLYGON (((297505.440 6543157.790, 297543..."
2,001.1A20,001,"MULTIPOLYGON (((297770.368 6543429.036, 297787..."
3,001.1A2A,001,"MULTIPOLYGON (((299678.370 6544460.320, 299667..."
4,001.1A2B,001,"MULTIPOLYGON (((303353.460 6552989.330, 303341..."


In [3]:
par_df = nivapy.da.get_nve_gts_api_parameters()
par_df

,Name,Fullname,NoDataValue,RawUnit,HumanReadableUnit,TimeResolutionInMinutes,FirstDateInTimeSerie
0,rr,Døgnnedbør v2.0,65535,mm,mm,1440,1957-01-01
1,tm,Døgntemperatur v2.0,65535,Kelvin,Celcius,1440,1957-01-01
4,swe,Snømengde v2.0.1,65535,mm,mm,1440,1957-01-01
5,snowload,Snølast,65535,kg/m2,kg/m2,1440,2000-01-01
6,sd,Snødybde v2.0.1,65535,mm,cm,1440,1957-01-01
7,swepr,Snømengde i prosent v2.0.1,65535,%,%,1440,1957-01-01
8,age,Snøens alder v2.0.1,255,Dager,Dager,1440,1957-01-01
9,qtt,Regn og snøsmelting v2.0.1,255,mm,mm,1440,1957-01-01
10,qtt7d,Regn og snøsmelting siste uke,65535,mm,mm,1440,1957-01-01
11,gwb_q,Avrenning,65535,mm,mm,1440,1957-01-01


In [5]:
%%capture

pars = ["rr", "tm", "gwb_q"]
st_dt = "1990-01-01"
end_dt = "2022-12-31"

reg_gdf = reg_gdf.head(50)

df_list = []
for idx, _ in tqdm(reg_gdf.iterrows(), total=reg_gdf.shape[0]):
    gdf = reg_gdf.loc[[idx]]
    try:
        res_df = nivapy.da.get_nve_gts_api_aggregated_time_series(
            gdf,
            pars,
            st_dt,
            end_dt,
            id_col="regine",
        )
        df_list.append(res_df)
    except (HTTPError, ValueError):
        pass
    
res_df = pd.concat(df_list, axis='rows')

In [9]:
res_df

,regine,par,datetime,mean_altitude_m,full_name,unit,time_resolution,value_min,value_median,value_max,value_mean,value_std,value_count
0,001.1A1,gwb_q,1990-01-01 06:00:00,50.0,Avrenning,mm,1440,1.80,1.80,1.80,1.80,NaN,1
1,001.1A1,gwb_q,1990-01-02 06:00:00,50.0,Avrenning,mm,1440,1.60,1.60,1.60,1.60,NaN,1
2,001.1A1,gwb_q,1990-01-03 06:00:00,50.0,Avrenning,mm,1440,1.40,1.40,1.40,1.40,NaN,1
3,001.1A1,gwb_q,1990-01-04 06:00:00,50.0,Avrenning,mm,1440,1.30,1.30,1.30,1.30,NaN,1
4,001.1A1,gwb_q,1990-01-05 06:00:00,50.0,Avrenning,mm,1440,1.10,1.10,1.10,1.10,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36154,001.B1A,tm,2022-12-27 06:00:00,114.0,Døgntemperatur v2.0,Celcius,1440,2.15,2.15,2.15,2.15,NaN,1
36155,001.B1A,tm,2022-12-28 06:00:00,114.0,Døgntemperatur v2.0,Celcius,1440,-0.85,-0.85,-0.85,-0.85,NaN,1
36156,001.B1A,tm,2022-12-29 06:00:00,114.0,Døgntemperatur v2.0,Celcius,1440,-1.25,-1.25,-1.25,-1.25,NaN,1
36157,001.B1A,tm,2022-12-30 06:00:00,114.0,Døgntemperatur v2.0,Celcius,1440,4.65,4.65,4.65,4.65,NaN,1
